In [22]:
# Sleeps are used because these notebooks are being tested automatically as part of the CI/CD. 
# In a normal user interaction, these sleeps would not be necessary.
from time import sleep

## Define a simple MLFlow workflow

In [1]:
def run_mlflow_workflow(sqlite_path, batch_size=64, number_epochs=10):
    import uuid
    import mlflow

    mlflow.set_tracking_uri(
        f"sqlite:///" f"{sqlite_path}"
    )
    experiment_name = "LinearRegression"
    experiment_id = mlflow.create_experiment(
        experiment_name + str(uuid.uuid4())
    )
    with mlflow.start_run(experiment_id=experiment_id) as run:
        mlflow.log_params({"number_epochs": number_epochs})
        mlflow.log_params({"batch_size": batch_size})

        print("Generated training metadata.")
        mlflow.log_metric("loss", 0.04)
        return run.info.run_uuid

In [2]:
# Set log levels
import os
os.environ['LOG_STREAM_LEVEL'] = "error"
os.environ['LOG_FILE_LEVEL'] = "debug"

## Initialize interceptor

In [3]:
from flowcept import MLFlowInterceptor
interceptor = MLFlowInterceptor()

## Initialize consumer API

In [4]:
from flowcept import FlowceptConsumerAPI
consumer = FlowceptConsumerAPI(interceptor)
consumer.start()

## Run MLFlow workflow

In [5]:
mlflow_run_id = run_mlflow_workflow(interceptor.settings.file_path)
print(f"MLflow task id={mlflow_run_id}")

Generated training metadata.
MLflow task id=5bcee668183949909cbb01ba1cea018c


In [ ]:
sleep(10)

## Initialize Query API

In [6]:
from flowcept import TaskQueryAPI
query_api = TaskQueryAPI()

## Query the task executed

In [7]:
_filter = {"task_id": mlflow_run_id}
query_api.query(_filter)

[{'task_id': '5bcee668183949909cbb01ba1cea018c',
  'utc_timestamp': 1676061860.004198,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True}]

# Inspecting more tasks through the query api

In [8]:
from flowcept.commons.utils import get_utc_minutes_ago

### Get the tasks executed in my experiment in the last 60 minutes

In [9]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment"
}
query_api.query(_filter)

[{'task_id': 'dummy_func1-ff697be9ec8641033d80d8f01e72b52b',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:55960',
   'scheduler_id': 'Scheduler-7cd46b5a-0f7a-40f1-85a9-b403b10d93b8',
   'scheduler_pid': 33398},
  'status': 'SUBMITTED',
  'utc_timestamp': 1676056235.084591,
  'activity_id': 'dummy_func1',
  'used': {'arg0': 0.2980732034559712},
  'workflow_id': 'wf_0d5e4a40-03e6-47ee-9f46-a047a47dfd19',
  'plugin_id': 'dask',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True,
  'address': 'tcp://127.0.0.1:55967',
  'generated': {'arg0': 0.5961464069119424},
  'start_time': 1676038201.8871877,
  'end_time': 1676038201.8871946,
  'dependents': ['dummy_func2-7602538d2e41e97b798f9289735e3e8c',
   'dummy_func3-adda83d53183fb96c2c7abb330fb5b20']},
 {'task_id': 'dummy_func2-7602538d2e41e97b798f9289735e3e8c',
  'custom_me

### Get the tasks executed in my experiment in the last 60 minutes that were executed using mlflow and dask

In [10]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment",
    "plugin_id": {"$in": ['mlflow', 'dask']}
}
docs = query_api.query(_filter)
docs

[{'task_id': 'dummy_func1-ff697be9ec8641033d80d8f01e72b52b',
  'custom_metadata': {'scheduler': 'tcp://127.0.0.1:55960',
   'scheduler_id': 'Scheduler-7cd46b5a-0f7a-40f1-85a9-b403b10d93b8',
   'scheduler_pid': 33398},
  'status': 'SUBMITTED',
  'utc_timestamp': 1676056235.084591,
  'activity_id': 'dummy_func1',
  'used': {'arg0': 0.2980732034559712},
  'workflow_id': 'wf_0d5e4a40-03e6-47ee-9f46-a047a47dfd19',
  'plugin_id': 'dask',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True,
  'address': 'tcp://127.0.0.1:55967',
  'generated': {'arg0': 0.5961464069119424},
  'start_time': 1676038201.8871877,
  'end_time': 1676038201.8871946,
  'dependents': ['dummy_func2-7602538d2e41e97b798f9289735e3e8c',
   'dummy_func3-adda83d53183fb96c2c7abb330fb5b20']},
 {'task_id': 'dummy_func2-7602538d2e41e97b798f9289735e3e8c',
  'custom_me

### Get the tasks executed in my experiment in the last 60 minutes that generated a batch_size > 0

In [11]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment",
    "plugin_id": 'dask',
    "generated.batch_size": { "$gte" : 0 }
}
docs = query_api.query(filter=_filter)
batch_sizes = []
for doc in docs:
    print(f"task={doc['task_id']}, generated batch_size={doc['generated']['batch_size']}")
    batch_sizes.append(doc['generated']['batch_size'])
batch_sizes

task=dummy_func3-adda83d53183fb96c2c7abb330fb5b20, generated batch_size=17
task=dummy_func3-a5532dbb19f6ee8eb9df16d34fe29101, generated batch_size=20


[17, 20]

### Now run a new MLFlow task using the batch_sizes generated by the Dask workflow

In [12]:
for batch_size in batch_sizes:
    mlflow_task = run_mlflow_workflow(interceptor.settings.file_path, batch_size=batch_size)
    print(mlflow_task)

Generated training metadata.
32ef81f60fa545bf8e3df81afa3a3a50
Generated training metadata.
4be8fd1616b54cb28a5812c5509c9585


In [19]:
sleep(10)

### Get these tasks

In [20]:
_filter = {
    "task_id": mlflow_task
}
docs = query_api.query(filter=_filter)
docs

[{'task_id': '4be8fd1616b54cb28a5812c5509c9585',
  'utc_timestamp': 1676061930.027624,
  'status': 'FINISHED',
  'used': {'batch_size': '20', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True}]

## Stopping consumers

In [21]:
consumer.stop()